In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

MP_API_KEY = os.getenv("MP_API_KEY")

In [2]:
from pymatgen.core import Structure
from pymatgen.io.vasp.sets import MPStaticSet

In [3]:
from mp_api.client import MPRester

with MPRester() as mpr:

    docs = mpr.materials.summary.search(
        material_ids=["mp-5777"],
        fields=["structure", "material_id"]
    )
    

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
import yaml
from pathlib import Path
from atomate2.settings import Atomate2Settings
from atomate2.vasp.jobs.core import RelaxMaker
from jobflow import run_locally
from pymatgen.io.vasp.sets import MatPESStaticSet
from pymatgen.io.vasp import Kpoints

structure = docs[0].structure

atomate2_config_file = Path(
    f"~/atomate2/config/atomate2-test.yaml"
).expanduser()
atomate2_config_file.parent.mkdir(parents=True, exist_ok=True)

vasp_prefix = Path("")
num_processes = 4

with open(atomate2_config_file, "w") as f:
    yaml.dump(
        {
            "VASP_CMD": f"mpirun -np {num_processes} {vasp_prefix / 'vasp_std'}",
            "VASP_GAMMA_CMD": f"mpirun -np {num_processes} {vasp_prefix / 'vasp_gam'}",
        },
        f,
    )

# pre_run = """
# module load nvhpc/23.11;
# export LD_LIBRARY_PATH=/global/software/rocky-8.x86_64/gcc/linux-rocky8-x86_64/gcc-8.5.0/nvhpc-23.11-gh5cygvdqksy6mxuy2xgoibowwxi3w7t/Linux_x86_64/23.11/compilers/extras/qd/lib:/global/software/rocky-8.x86_64/gcc/linux-rocky8-x86_64/gcc-8.5.0/nvhpc-23.11-gh5cygvdqksy6mxuy2xgoibowwxi3w7t/Linux_x86_64/23.11/cuda/lib64:$LD_LIBRARY_PATH
# """

# pre_run = """export LD_LIBRARY_PATH=/global/software/rocky-8.x86_64/gcc/linux-rocky8-x86_64/gcc-8.5.0/nvhpc-23.11-gh5cygvdqksy6mxuy2xgoibowwxi3w7t/Linux_x86_64/23.11/compilers/extras/qd/lib:/global/software/rocky-8.x86_64/gcc/linux-rocky8-x86_64/gcc-8.5.0/nvhpc-23.11-gh5cygvdqksy6mxuy2xgoibowwxi3w7t/Linux_x86_64/23.11/cuda/lib64:$LD_LIBRARY_PATH"""

os.environ["LD_LIBRARY_PATH"] = "/global/software/rocky-8.x86_64/gcc/linux-rocky8-x86_64/gcc-8.5.0/nvhpc-23.11-gh5cygvdqksy6mxuy2xgoibowwxi3w7t/Linux_x86_64/23.11/compilers/extras/qd/lib:/global/software/rocky-8.x86_64/gcc/linux-rocky8-x86_64/gcc-8.5.0/nvhpc-23.11-gh5cygvdqksy6mxuy2xgoibowwxi3w7t/Linux_x86_64/23.11/cuda/lib64:$LD_LIBRARY_PATH"


settings = Atomate2Settings(
    CONFIG_FILE=str(atomate2_config_file),
    VASP_CMD=f"mpirun -np {num_processes} {vasp_prefix / 'vasp_std'}",
    VASP_GAMMA_CMD=f"mpirun -np {num_processes} {vasp_prefix / 'vasp_gam'}",
)

input_set_generator = MatPESStaticSet(
    user_kpoints_settings=Kpoints(), #{"reciprocal_density": 10},
)

relax_job = RelaxMaker(
    input_set_generator=input_set_generator,
    run_vasp_kwargs=dict(
        vasp_cmd=settings.VASP_CMD,
        vasp_gamma_cmd=settings.VASP_GAMMA_CMD,
    ),
).make(structure)


# run the job
run_locally(relax_job, create_folders=True)


2025-02-02 23:17:19,563 INFO Started executing jobs locally
2025-02-02 23:17:19,583 INFO Starting job - relax (06e7e0cd-64b6-4ee8-ab13-1cdb8d870df5)


/global/home/users/cyrusyc/.conda/envs/mlip-arena/lib/python3.12/site-packages/custodian/vasp/handlers.py:185: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(os.path.join(directory, self.output_filename), mode="rt") as file:


2025-02-02 23:18:25,305 INFO Finished job - relax (06e7e0cd-64b6-4ee8-ab13-1cdb8d870df5)
2025-02-02 23:18:25,311 WARNING Response.stored_data is not supported with local manager.
2025-02-02 23:18:25,323 INFO Finished executing jobs locally


{'06e7e0cd-64b6-4ee8-ab13-1cdb8d870df5': {1: Response(output=TaskDoc(builder_meta=EmmetMeta(emmet_version='0.84.6rc3', pymatgen_version='2025.1.24', run_id=None, batch_id=None, database_version=None, build_date=datetime.datetime(2025, 2, 3, 7, 18, 21, 810031, tzinfo=datetime.timezone.utc), license=None), nsites=5, elements=[Element Ba, Element O, Element Ti], nelements=3, composition=Composition('Ba1 Ti1 O3'), composition_reduced=Composition('Ba1 Ti1 O3'), formula_pretty='BaTiO3', formula_anonymous='ABC3', chemsys='Ba-O-Ti', volume=65.57310741671338, density=5.905237274567591, density_atomic=13.114621483342678, symmetry=SymmetryData(crystal_system=<CrystalSystem.ortho: 'Orthorhombic'>, symbol='Amm2', number=38, point_group='mm2', symprec=0.1, angle_tolerance=5.0, version='2.5.0'), tags=None, dir_name='n0026.es1:/global/scratch/users/cyrusyc/benchmarks/lawrencium/vasp/job_2025-02-03-07-17-19-578646-74389', state=<TaskState.SUCCESS: 'successful'>, calcs_reversed=[Calculation(dir_name='/g